# リバーシを作る

## 仕様
- 必須機能
  - 以下に示すような画面例を持つリバーシをプレイできる。（リバーシのルールは検索して調べること）
  - 人vs人対戦
  - 入力はキーボードからの数字とアルファべット→input()?
  - ゲーム終了時に勝敗，白と黒の石の数を表示
- 任意機能
  - 人vs ＣＰＵモード、CPU vs CPUモードの追加
  - 対戦結果の外部ファイルへの出力
  - ゲーム途中のセーブ機能とロード機能
  - その他の機能を自由に追加して構わない

## メモ
枠をつくる(8*8)
中央2*2[4,4], [4,5], [5,4], [5,5]の
○●
●○←が必要

白と黒の石（○と●）
挟んだら色が反転する
相手の色を挟める場所しか置けない
挟めなかったらパス
全部埋まるor どっちも置けない→ゲーム終了

### 作る関数
1. 盤面を表示
2. 石を置く
3. 石が置けるかをチェック

1. board関数
   tablesize*tablesizeの配列をつくる（intにする）
   初期条件：[3][3],[4][4]に白，[3][4]，[4][3]に黒
3. display関数
   配列の各要素に白，黒，空白を割り当て8*8できれいに表示
4. check関数
   盤内においたか
   →周りの8マスに相手の石があるか
    ない→faulse
    ある→その直線上の石を調べる
        自分の石→faulse
        相手の石→loop
5. put関数
   置く座標と色を指定
   →board[x][y] = colorとする
            
    

In [ ]:
import numpy as np

white = 1
black = -1
blank = 0
tablesize = 8

class Reversi(object):
    
    def __init__(self):
        self.cell = np.zeros((tablesize, tablesize))
        self.cell = self.cell.astype(int)
        self.cell[3][3] = 1
        self.cell[3][4] = -1
        self.cell[4][3] = -1
        self.cell[4][4] = 1
        self.current = black    # 置いたプレイヤー
        self.turn = 1
        
    def turn_change(self):
        self.current *= -1
    
    def display(self):
        print("-" * 20)
        for x in range(tablesize):
            for y in range(tablesize):
                if self.cell[x][y] == 0:
                    print("-", end="  ")
                elif self.cell[x][y] == 1:
                    print("○", end="  ")
                else:
                    print("●", end="  ")
            print()

    def put(self, x, y):    # あとで1ターンの処理をまとめる
        x = int(input("xを入力"))
        y = int(input("yを入力"))
        color = int(input("1 or -1"))
        
        self.cell[x][y] = color
        
    # 10/30ここまで
    # ここから→プレイヤーを分ける，石を置けるかチェックする，石を反転する

    def range_check(self, x, y):    # 盤内チェック
        return 0 <= x < tablesize and 0 <= y < tablesize
    
    def put_check(self, x, y):  # おけるかチェック
        if self.range_check(x, y):
            if self.cell[x][y] == blank:
                if self.flip_stone(x, y):
                    return True
        return False
    
    def flip_check(self, x, y, dx, dy): # 反転チェック
        if self.range_check(x+dx, y+dy):
            while self.cell[x+dx][y+dy] == -self.current:
                
        
            
            
    
    
    
if __name__ == "__main__":
    board = Reversi()
    board.display()
    board.put(3, 2)
# b = board(8)
# b[3][2] = 1
# print(b)


# display(b)

# # put追加hhhhhhhhhhh
# put()
# range_check()
# display(b)

--------------------
-  -  -  -  -  -  -  -  
-  -  -  -  -  -  -  -  
-  -  -  -  -  -  -  -  
-  -  -  ○  ●  -  -  -  
-  -  -  ●  ○  -  -  -  
-  -  -  -  -  -  -  -  
-  -  -  -  -  -  -  -  
-  -  -  -  -  -  -  -  


TypeError: Reversi.put() missing 2 required positional arguments: 'x' and 'y'